In [9]:
#!/usr/bin/env python3

import pandas as pd
import errno    
import os
import numpy as np
import math

'''
Author: Enrico Ceccolini
    TODO write the description
'''

#datadir = "/datasets/eurora_data/db1/"
#datadir = "/datasets/eurora_data/db/"
datadir = "C:/Users/folid/git-repo/hpc_job_power_prediction/datasets/eurora_data/db_local/"
infile_jobs_to_nodes = datadir + "job_nodes.csv"


infile_nodes = datadir + "nodes.csv"

suffix = "_5sec_"
### select an interval from
## 1 settings wholeData
#interval_comment_whole = "WholeData"

### select an interval from
## 2 settings Andrea
#interval_comment = "Andrea"
#start_time = pd.to_datetime('2014-03-31')
#end_time = pd.to_datetime('2014-05-01')
#infile_jobs = datadir + "CPUs/" + interval_comment + "/jobs_cleaned"

## 3 settings Alina
interval_comment = "Alina"
start_time = pd.to_datetime('2014-06-30')
end_time = pd.to_datetime('2014-11-01')
infile_jobs = datadir + "CPUs/" + interval_comment + "/" + interval_comment + "_long_jobs_real_pow"

nodes=['01', '02', '03', '04', '05', '06', '07', '08', '09', '10', '11', '12', '13', '14', '15', '16', '17', '18', '19', '20', '21', '22', '23', '24', '25', '26', '27', '28', '29', '30', '31', '32', '33', '34', '35', '36', '37', '38', '39', '40', '41', '42', '44', '45', '46', '47', '48', '49', '50', '51', '52', '53', '54', '55', '56', '57', '58', '59', '60', '61', '62', '63', '64']


Along the nodes we have three differentIntel Xeon E5 processors stepping: nodes 1-16 & 25-32 havea maximum frequency of 2.1GHz (CPUs-2100), nodes 17-24 have a maximum frequency of 2.2GHz (CPUs-2200), andnodes 33-64 have a maximum frequency of 3.1GHz (CPUs-3100).

In [2]:
nodes_data = pd.read_csv(infile_nodes, index_col=0)
# nodes_data

### Read job2nodes data

In [3]:
jobs_to_nodes_whole_data = pd.read_csv(infile_jobs_to_nodes, index_col=0)
print("jobs_to_nodes_whole_data contains {} records".format(jobs_to_nodes_whole_data.shape[0]))

jobs_to_nodes_whole_data contains 469095 records


#### solve the problem of jobs runned on node 129

In [4]:
jobs_to_nodes_whole_data = jobs_to_nodes_whole_data[jobs_to_nodes_whole_data['node_id'] != 129]
print("jobs_to_nodes_whole_data now contains {} records".format(jobs_to_nodes_whole_data.shape[0]))

jobs_to_nodes_whole_data now contains 469020 records


#### solve the problem of duplicates job_id_string, node_id
this is probably caused by the concurrency writing of the database 

In [5]:
jobs_to_nodes_whole_data = jobs_to_nodes_whole_data.drop_duplicates(subset=['job_id_string', 'node_id'])
print("jobs_to_nodes_whole_data now contains {} records".format(jobs_to_nodes_whole_data.shape[0]))

jobs_to_nodes_whole_data now contains 444610 records


### Read jobs data

drop the jobs out of interval

In [16]:
jobs = pd.read_csv(infile_jobs + ".csv", index_col=0)
print("jobs_whole_data contains {} records".format(jobs.shape[0]))

C:\Users\folid\Anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3049: DtypeWarning: Columns (11) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


jobs_whole_data contains 336787 records


In [21]:
jobs = jobs[pd.to_datetime(jobs['end_time']) <= end_time]
jobs = jobs[pd.to_datetime(jobs['run_start_time']) > start_time]
print("Alina's interval contains {} records".format(jobs.shape[0]))

Alina's interval contains 237428 records


#### merge

In [22]:
merged_jobs_to_nodes = pd.merge(jobs_to_nodes_whole_data, jobs, on='job_id_string')
print(merged_jobs_to_nodes.shape[0])

252284


### add an approximation of the power consumed on completely missing nodes

In [23]:
jobs[jobs['good_nodes'] != jobs['n_2_1'] + jobs['n_2_2'] + jobs['n_3_1']].shape[0]

5284

In [29]:
jobs[jobs['good_nodes'] != jobs['node_req']].shape[0]

5284

In [30]:
jobs_to_adjust = jobs[jobs['good_nodes'] != jobs['node_req']]

In [31]:
for index_job, row_job in jobs_to_adjust.iterrows():
    tot_nodes = row_job['node_req']
    real_pow = row_job['real_pow']
    
    real_pow += (1 - row_job['good_nodes'] / tot_nodes) * real_pow
    
    jobs.at[index_job, 'real_pow'] = real_pow

In [35]:
#debug
#jobs_to_adjust = jobs[jobs['good_nodes'] != jobs['node_req']]
#jobs_to_adjust[jobs_to_adjust['real_pow'] > 200]

### calculate the job pow approximation quality

In [36]:
jobs['real_pow_quality']  = 1 - ((jobs['job_tot_timepoints'] - jobs['job_timepoints']) / jobs['job_tot_timepoints'])

In [41]:
jobs.sort_values('job_timepoints')

,job_id_string,job_name,queue,start_time,run_start_time,end_time,user,node_req,cpu_req,mem_req,...,mic_req,real_pow,ran_alone,real_pow_quality,n_2_1,n_2_2,n_3_1,job_tot_timepoints,job_timepoints,good_nodes
job_id,,,,,,,,,,,,,,,,,,,,,
205721,958010.node129,acyclovir9,R928065,2014-08-11 08:42:56,2014-08-11 08:42:56,2014-08-11 08:43:02,sdecherc@node046.eurora.cineca.it,1,16,8388608,...,0.0,55.828818,True,0.000000,0,0,1,1,0,1
290179,1279299.node129,SMD_4h_4,parallel,2014-09-26 06:43:51,2014-09-26 06:43:52,2014-09-26 06:43:58,szia0000@node056.eurora.cineca.it,1,2,8388608,...,0.0,6.956499,True,0.000000,0,0,1,1,0,1
290178,1279298.node129,SMD_4h_1,parallel,2014-09-26 06:43:49,2014-09-26 06:43:50,2014-09-26 06:43:56,szia0000@node044.eurora.cineca.it,1,2,8388608,...,0.0,4.748908,True,0.000000,0,0,1,1,0,1
290177,1279297.node129,SMD_4h_8,parallel,2014-09-26 06:43:48,2014-09-26 06:43:49,2014-09-26 06:43:54,szia0000@node045.eurora.cineca.it,1,2,8388608,...,0.0,55.750079,True,0.000000,0,0,1,1,0,1
290176,1279296.node129,SMD_4h_7,parallel,2014-09-26 06:43:47,2014-09-26 06:43:48,2014-09-26 06:43:56,szia0000@node035.eurora.cineca.it,1,2,8388608,...,0.0,23.333252,True,0.000000,0,0,1,1,0,1
290175,1279295.node129,SMD_4h_9,parallel,2014-09-26 06:43:46,2014-09-26 06:43:47,2014-09-26 06:43:54,szia0000@node063.eurora.cineca.it,1,2,8388608,...,0.0,72.764636,True,0.000000,0,0,1,1,0,1
290174,1279294.node129,SMD_4h_4,parallel,2014-09-26 06:43:44,2014-09-26 06:43:45,2014-09-26 06:43:51,szia0000@node056.eurora.cineca.it,1,2,8388608,...,0.0,7.722827,True,0.000000,0,0,1,1,0,1
290173,1279293.node129,SMD_4h_8,parallel,2014-09-26 06:43:41,2014-09-26 06:43:43,2014-09-26 06:43:48,szia0000@node044.eurora.cineca.it,1,2,8388608,...,0.0,58.675070,True,0.000000,0,0,1,1,0,1
290172,1279292.node129,SMD_4h_1,parallel,2014-09-26 06:43:40,2014-09-26 06:43:43,2014-09-26 06:43:49,szia0000@node045.eurora.cineca.it,1,2,8388608,...,0.0,27.045405,True,0.000000,0,0,1,1,0,1


In [42]:
jobs.to_csv(infile_jobs + ".csv")

In [ ]:
# other file code SVI

In [ ]:
#svi
#jobs_whole[jobs_whole['job_id_string'] == '1006691.node129'].index

In [ ]:
i = 0
outfile = datadir + 'CPUs/' + interval_comment + "/" + interval_comment + "_long_jobs_real_pow"

for node in nodes:
    merged_jobs_to_node = merged_jobs_to_nodes[merged_jobs_to_nodes['node_id'] == int(node)]
    print("---------- {} jobs in node {}".format(merged_jobs_to_node.shape[0], node))
    infile_node = datadir + "CPUs/" + interval_comment + "/node" + node + suffix + interval_comment + "_active_cores_and_jobs.csv"
    node_measurements = pd.read_csv(infile_node)
    node_start_time = node_measurements['timestamp'].iloc[0]
    num_minutes = node_measurements.shape[0]
    print("-------------------node data starts at {}, num minute {}".format(node_start_time, num_minutes))
    
    for index_job_to_node, row_job_to_node in merged_jobs_to_node.iterrows():
        print()
        print("{}/{}".format(i, merged_jobs_to_nodes.shape[0]))
        
        partial_pow_consumption = 0
        job_id_string = row_job_to_node['job_id_string']
        
        # recupero indice del job in jobs_whole
        job = jobs_whole[jobs_whole['job_id_string'] == job_id_string]
        job_index = jobs_whole[jobs_whole['job_id_string'] == job_id_string].index
        if(job_index != None):
            job_real_consumption = job['real_pow'].iloc[0]
            job_n_2_1 = job['n_2_1'].iloc[0]
            job_n_2_2 = job['n_2_2'].iloc[0]
            job_n_3_1 = job['n_3_1'].iloc[0]

            job_tot_timepoints = job['job_tot_timepoints'].iloc[0]
            job_timepoints = job['job_timepoints'].iloc[0]
            job_good_nodes = job['good_nodes'].iloc[0]
            job_ran_alone = job['ran_alone'].iloc[0]

            node_type = nodes_data.iloc[row_job_to_node['node_id']-1]['cpu_type']
            if(node_type == '2_1_ghz'):
                job_n_2_1 += 1
            elif(node_type == '2_2_ghz'):
                job_n_2_2 += 1
            elif(node_type == '3_1_ghz'):
                job_n_3_1 += 1

            node_idle_core = nodes_data.iloc[row_job_to_node['node_id']-1]['core_idle']
            
            job_start_time = pd.to_datetime(row_job_to_node['run_start_time'])
            job_end_time = pd.to_datetime(row_job_to_node['end_time'])
            
            if(job_end_time - job_start_time >= np.timedelta64(5, 's')):
                before_minutes = int((job_start_time - pd.to_datetime(node_start_time)) / np.timedelta64(5, 's'))
                running_minutes = int((job_end_time - job_start_time) / np.timedelta64(5, 's'))
                after_minutes = num_minutes - running_minutes - before_minutes
                
                before_serie = pd.Series(False, index=np.arange(before_minutes))
                running_serie = pd.Series(True, index=np.arange(running_minutes))
                after_serie = pd.Series(False, index=np.arange(after_minutes))
                concat_series = pd.concat([before_serie, running_serie, after_serie], ignore_index=True)
                
                interval_measurements = node_measurements[concat_series]
                
                ### take only the entries of the interval where the job was running
                #interval_measurements = node_measurements.loc[(pd.to_datetime(node_measurements['timestamp']) >= pd.to_datetime(row_job_to_node['run_start_time'])) & (pd.to_datetime(node_measurements['timestamp']) <= pd.to_datetime((row_job_to_node['end_time'])) - np.timedelta64(5, 's'))]
                ### drop missing measurements (calculate the percentage)
                n_node_measurements = interval_measurements.shape[0]
                job_tot_timepoints += n_node_measurements

                interval_measurements = interval_measurements.dropna()
                print("{}/{} intervals missing".format(n_node_measurements-interval_measurements.shape[0], n_node_measurements))
                job_timepoints += n_node_measurements-interval_measurements.shape[0]

                ### drop where active cores is greater than 16 - this problem occours only once on node 30
                interval_measurements = interval_measurements[interval_measurements['active_cores'] <= 16]
                ### drop row with 0 active_cores (or less than the current job used cores)
                interval_measurements = interval_measurements[interval_measurements['active_cores'] >= row_job_to_node['ncpus']]  

                if(interval_measurements.shape[0] != 0):
                    job_good_nodes += 1 


                ### group the intervals wehere the partial_pow_cons can be obtaied with the same instance of the formula
                ### take the mean for the pow columns
                # interval_grouped = interval_measurements.reset_index().groupby(["active_cores", "active_jobs", "active_gpus", "active_mics"]).mean()
                interval_grouped = interval_measurements.groupby(["active_cores", "active_jobs", "active_gpus", "active_mics"])
                counts = interval_grouped.size().to_frame(name='counts')
                interval_grouped = (counts
                 .join(interval_grouped.agg({'pow_tot':'mean'}).rename(columns={'pow_tot': 'pow_tot_mean'}))
                 .reset_index()
                )
                interval_grouped.sort_values('active_jobs')

                if(interval_grouped.shape[0]>1):
                    job_ran_alone = False


                if(interval_grouped.shape[0] != 0):
                    #interval_grouped['pow_tot'] = (interval_grouped['pow_tot_0_mean'] + interval_grouped['pow_tot_1_mean']) * row['ncpus'] / interval_grouped['active_cores'] 
                    interval_grouped['pow_tot'] = (interval_grouped['pow_tot_mean'] - (16 - interval_grouped['active_cores'])*node_idle_core) * row_job_to_node['ncpus'] / interval_grouped['active_cores'] 

                    # not_alone_counts = interval_grouped['counts'].loc[interval_grouped['active_jobs'] != 1].sum()
                    # interval_grouped.loc[interval_grouped['active_jobs'] == 1, ['counts']] += not_alone_counts
                    print(interval_grouped)
                    partial_pow_consumption = np.average(interval_grouped['pow_tot'], weights=interval_grouped['counts'])
                    # partial_pow_consumption = partial_pow_consumption / interval_grouped.shape[0]
                    # print(interval_grouped)

                print("partial measurement: {}".format(partial_pow_consumption))
                job_real_consumption += partial_pow_consumption
                partial_pow_consumption = 0

        
            else: # jobs shorter than 5 sec
                   # keep the only timepoint
                measure = node_measurements[node_measurements['timestamp'] == row_job_to_node['run_start_time'][:-3]+':00']
                job_tot_timepoints += 1
                if(measure.shape[0] != 0):
                    job_timepoints += 1
                    job_good_nodes += 1
                    if(measure['active_cores'].iloc[0] != 0):
                        job_ran_alone = False

                    active_cores = measure['active_cores'].iloc[0] + row_job_to_node['ncpus']
                    if(active_cores > 16):
                        active_cores = 16
                    job_real_consumption += (measure['pow_tot'].iloc[0] - (16 - active_cores) * node_idle_core) * row_job_to_node['ncpus'] / active_cores
                else:
                    print("error measure")
                    job_pow_quality = 0.0
               
            
            if(job_real_consumption < 0):
                job_real_consumption = 0.0
            print("job_real_consumption: {}".format(job_real_consumption))
            
            jobs_whole.at[job_index, 'real_pow'] = job_real_consumption
            jobs_whole.at[job_index, 'n_2_1'] = job_n_2_1
            jobs_whole.at[job_index, 'n_2_2'] = job_n_2_2
            jobs_whole.at[job_index, 'n_3_1'] = job_n_3_1
            jobs_whole.at[job_index, 'job_tot_timepoints'] = job_tot_timepoints
            jobs_whole.at[job_index, 'job_timepoints'] = job_timepoints
            jobs_whole.at[job_index, 'good_nodes'] = job_good_nodes
            jobs_whole.at[job_index, 'ran_alone'] = job_ran_alone
    
        
        i = i + 1
    #if(i % 10000 == 0):
    jobs_whole.to_csv(outfile + "_" + node + "_" + str(i) + ".csv")
        
jobs_whole.to_csv(outfile + "_" + node + "_" + str(i) + ".csv")